In [ ]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [ ]:
# model = LpProblem(name="small-problem", sense=LpMaximize)

# # Initialize the decision variables
# x = LpVariable(name="x", lowBound=0)
# y = LpVariable(name="y", lowBound=0)

# model += (2 * x + y <= 20, "red_constraint")
# model += (4 * x - 5 * y >= -10, "blue_constraint")
# model += (-x + 2 * y >= -2, "yellow_constraint")
# model += (-x + 5 * y == 15, "green_constraint")

# model += x + 2 * y

# status = model.solve()

In [ ]:
# print(f"status: {model.status}, {LpStatus[model.status]}")
# print(f"objective: {model.objective.value()}")
# for name, constraint in model.constraints.items():
#      print(f"{name}: {constraint.value()}")

In [ ]:
# model = LpProblem(name="small-problem", sense=LpMaximize)

# # Initialize the decision variables
# x1 = LpVariable(name="x1", lowBound=0)
# y1 = LpVariable(name="y1", lowBound=0)

# model += (2 * x1 + y1 <= 20, "red_constraint")
# model += (-x1 + 5 * y1 == 15, "green_constraint")

# model += x1 + 2 * y1

# status = model.solve()

In [ ]:
# Define the model
model = LpProblem(name="resource-allocation", sense=LpMaximize)

constraints = {
    '0-10': ((1, 10), 213, 826.00),
    '2-3': ((2, 2), 159, 437.80)
}

variables_n = { i: [] for i in range(1, 10) }
variables_w = { i: [] for i in range(1, 10) }

for constraint_name, constraint in constraints.items():
    range_buckets, range_count, range_total_weight = constraint
    # Define the decision variables
    an = {i: LpVariable(name=f"{constraint_name}_n_{i}", lowBound=0) for i in range(range_buckets[0], range_buckets[1])}
    aw = {i: LpVariable(name=f"{constraint_name}_w_{i}", lowBound=0) for i in range(range_buckets[0], range_buckets[1])}

    # Add constraints
    model += (lpSum(an.values()) == range_count)
    model += (lpSum(aw.values()) == range_total_weight)
    for index in an:
        var = an[index]
        model += (aw[index] - index * an[index] >= 0)
        model += (aw[index] - (index + 1) * an[index] <= 0)
        variables_n[index].append(an[index])
        variables_w[index].append(aw[index])

for index in variables_n:
    model += (lpSum(variables_w[index]) - index * lpSum(variables_n[index]) >= 0)
    model += (lpSum(variables_w[index]) - (index + 1) * lpSum(variables_n[index]) <= 0)
        
# Set the objective
for index in variables_n:
    for index2, var in enumerate(variables_n[index]):
        print(index, index2)
        print(variables_w[index][0])
        print(variables_n[index][0])
        #model += (lpSum(variables_w[index]) * lpSum(variables_n[index]) - variables_w[index][index2] * variables_n[index][index2])
        model += (variables_w[index][0] * variables_n[index][0])
        
# Solve the optimization problem
status = model.solve()

# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in a_n.values():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

In [ ]:
# https://stackoverflow.com/questions/19664865/migrating-from-pulp-to-scipy

import numpy as np
import scipy.optimize as opt

constraints = {
    '0-10': ((2, 5), 213, 826.00),
    '2-3': ((2, 3), 159, 437.80),
    '3-10': ((3, 5), 667, 1667.2),
    '3-4': ((3, 4), 4089, 14856.00),
    '3-6': ((3, 6), 4279, 19030.50),
    '4.5-5.0': ((4, 5), 540, 2553.80),
    '4-5': ((4, 5), 14872, 67174.40),
    '5-6': ((5, 6), 14264, 77238.80),
    '6+': ((6, 8), 618, 4071.80),
    '6-7': ((6, 7), 4640, 29502.20),
    '7-8': ((7, 8), 730, 5323.00),
    '8-9': ((8, 9), 49, 402.20),
    '9+': ((9, 10), 2, 18.30),
}

all_n_indices = []
all_w_indices = []

variable_n_indices = {}
variable_w_indices = {}

index_to_bucket = {}

variables_n = { i: [] for i in range(1, 10) }
variables_w = { i: [] for i in range(1, 10) }

conditions = []
bounds = []
guess = []

counter = 0

all_weights = 0
all_counts = 0

for constraint_name, constraint in constraints.items():
    range_buckets, range_count, range_total_weight = constraint
    
    all_weights = all_weights + range_total_weight / 10000
    all_counts = all_counts + range_count / 10000

    range_count = range_count / 10000
    range_total_weight = range_total_weight / 10000
    
    variable_n_indices[constraint_name] = []
    variable_w_indices[constraint_name] = []
    
    for bucket in range(range_buckets[0], range_buckets[1]):
        guess_n = range_count / (range_buckets[1] - range_buckets[0])
        guess_w = range_total_weight / (range_buckets[1] - range_buckets[0])
        
        variable_n_indices[constraint_name].append(counter)
        variables_n[bucket].append(counter)
        all_n_indices.append(counter)
        guess.append(guess_n)
        bounds.append((0, range_count))
        index_to_bucket[counter] = bucket

        counter = counter + 1
        
        variable_w_indices[constraint_name].append(counter)
        variables_w[bucket].append(counter)
        all_w_indices.append(counter)
        guess.append(guess_w)
        bounds.append((0, range_total_weight))
        index_to_bucket[counter] = bucket
        
        counter = counter + 1
    
    variable_n_indices[constraint_name] = np.array(variable_n_indices[constraint_name])
    variable_w_indices[constraint_name] = np.array(variable_w_indices[constraint_name])
    
    n_indices = variable_n_indices[constraint_name].astype(int)
    w_indices = variable_w_indices[constraint_name].astype(int)
    
#     print(n_indices)
#     print(w_indices)
    
    conditions.append({ 'type': 'ineq', 'fun': lambda x: 0.000001 - np.abs(np.sum(x[n_indices]) - range_count) })
    conditions.append({ 'type': 'ineq', 'fun': lambda x: 0.000001 - np.abs(np.sum(x[w_indices]) - range_total_weight) })
        
    for index, bucket in enumerate(range(range_buckets[0], range_buckets[1])):
        n_index = variable_n_indices[constraint_name][index]
        w_index = variable_w_indices[constraint_name][index]
#         print(bucket, n_index, w_index)

        x1 = min((bucket + 1) - x[w_index] / x[n_index], min(0, x[w_index] / x[n_index] - bucket))
        print(x1)
        
#         conditions.append({ 'type': 'ineq', 'fun': lambda x: min(0.000001, x[w_index] / x[n_index] - bucket)  })
#         conditions.append({ 'type': 'ineq', 'fun': lambda x: min(0.000001, (bucket + 1) - x[w_index] / x[n_index]) })
        conditions.append({ 'type': 'ineq', 'fun': lambda x: 0.000001 + x1 })
    
for bucket in variables_n:
    variables_n[bucket] = np.array(variables_n[bucket])
    variables_w[bucket] = np.array(variables_w[bucket])
    
    n_indices = variables_n[bucket].astype(int)
    w_indices = variables_w[bucket].astype(int)
    
    if len(n_indices) == 0:
        continue
    
#     print(bucket)
#     print('a', n_indices, np.sum(n_indices))
#     print('b', w_indices, np.sum(w_indices))
    
    conditions.append({ 'type': 'ineq', 'fun': lambda x: np.sum(x[w_indices]) - bucket * np.sum(x[n_indices]) })
    conditions.append({ 'type': 'ineq', 'fun': lambda x: (bucket + 1) * np.sum(x[n_indices]) - np.sum(x[w_indices]) })

conditions.append({ 'type': 'ineq', 'fun': lambda x: 0.000001 - np.abs(np.sum(x[all_n_indices]) - all_counts) })
conditions.append({ 'type': 'ineq', 'fun': lambda x: 0.000001 - np.abs(np.sum(x[all_w_indices]) - all_weights) })
    
def objective_function(x):
    objective = 0
    
    for bucket in variables_n:
        n_indices = variables_n[bucket].astype(int)
        w_indices = variables_w[bucket].astype(int)
        
        for index, n_index in enumerate(variables_n[bucket]):
            w_index = w_indices[index]
            
            x1 = bucket - (x[w_index] / x[n_index])
            x2 = (x[w_index] / x[n_index]) - (bucket + 1)
            
#             print(x1, x2)
#             print(x1, x2, 1 - .5 * (1 + x1 + np.sqrt((x1 - 1) ^ 2 + .01) ) + 1 - .5 * (1 + x2 + np.sqrt((x2 - 1) ^ 2 + .01) ))
            
#             objective = objective + .1 * (1 - .5 * (1 + x1 + np.sqrt((x1 - 1) ** 2 + .01) ) + 1 - .5 * (1 + x2 + np.sqrt((x2 - 1) ** 2 + .01) ))
#             objective = objective + .01 * (-np.log(1 + np.exp(-(-x1))) + -np.log(1 + np.exp(-(-x2))))
#             a = (-np.log(1 + np.exp(-(x[w_index] / x[n_index] - bucket))) + -np.log(1 + np.exp(-((bucket + 1) - x[w_index] / x[n_index]))))
#             if not np.isfinite(a):
#                 print(x[w_index], x[n_index], bucket, x[w_index] / x[n_index] - bucket)
#             print('a', n_indices, n_index, np.sum(x[n_indices]), x[n_index])
#             print('b', w_indices, w_index, np.sum(x[w_indices]), x[w_index])
#             print('c',(x[w_index] / x[n_index] - np.sum(x[w_indices]) / np.sum(x[n_indices])))
    
            # TODO: Remove this as I think it's not correct
#             objective = objective + np.abs((x[w_index] / x[n_index]) - (np.sum(x[w_indices]) / np.sum(x[n_indices])))
#             objective = objective + x[n_index] * np.abs((x[w_index] / x[n_index]) - (np.sum(x[w_indices]) / np.sum(x[n_indices])))
#             objective = objective + np.abs((x[n_index] / x[n_index]) - (np.sum(x[n_indices]) / all_counts))

    for constraint_name, constraint in constraints.items():
        range_buckets, range_count, range_total_weight = constraint
        
        range_count = range_count / 10000
        range_total_weight = range_total_weight / 10000
        
        n_indices = variable_n_indices[constraint_name].astype(int)
        w_indices = variable_w_indices[constraint_name].astype(int)
        
        total_subset_count = 0
        
        for n_index in n_indices:
            bucket = index_to_bucket[n_index]
            bucket_indices = variables_n[bucket]
            total_subset_count = total_subset_count + np.sum(x[bucket_indices])
        
        for index, n_index in enumerate(n_indices):
            w_index = w_indices[index]
            
            bucket = index_to_bucket[n_index]
            bucket_indices = variables_n[bucket]
            
            objective = objective + np.abs((x[n_index] / np.sum(n_indices) - np.sum(x[bucket_indices]) / total_subset_count))
            #objective = objective + x[n_index] * np.abs((x[n_index] / np.sum(n_indices) - np.sum(x[bucket_indices]) / total_subset_count))
        
        objective = objective + np.abs(np.sum(x[n_indices]) - range_count) / range_count
        objective = objective + np.abs(np.sum(x[w_indices]) - range_total_weight) / range_total_weight
    
    return objective
      
# print(guess)
# print(bounds)
# print(conditions)
print(all_weights, all_counts)
    
results = opt.minimize(objective_function, guess, method='SLSQP', bounds=bounds, constraints = conditions, options={'ftol': 1e-9, 'disp': True})
print(results)

In [ ]:
x = results.x

all_weight = 0
all_count = 0

for bucket in variables_n:
    n_indices = variables_n[bucket].astype(int)
    w_indices = variables_w[bucket].astype(int)
    
    total_weight = np.sum(x[w_indices]) * 10000
    total_count = np.sum(x[n_indices]) * 10000
    
    all_weight = all_weight + total_weight
    all_count = all_count + total_count

print('%0.2f vs %0.2f, %0.2f vs %0.2f' % (all_weight / 10000, all_weights, all_count / 10000, all_counts))

print()
    
for bucket in variables_n:
    n_indices = variables_n[bucket].astype(int)
    w_indices = variables_w[bucket].astype(int)
    
    total_weight = np.sum(x[w_indices]) * 10000
    total_count = np.sum(x[n_indices]) * 10000
    
    print('%i - %i: %0.2f, %0.2f' % (bucket * 1000, (bucket + 1) * 1000, total_weight / total_count * 1000, total_count / all_count * 100))

print()
    
for constraint in variable_n_indices:
    n_indices = variable_n_indices[constraint].astype(int)
    w_indices = variable_w_indices[constraint].astype(int)
    
    total_weight = np.sum(x[w_indices]) * 10000
    total_count = np.sum(x[n_indices]) * 10000
    
    range_buckets, range_count, range_total_weight = constraints[constraint]
    
    abs_weight_diff = np.abs((range_total_weight - total_weight) / range_total_weight) * 100
    abs_count_diff = np.abs((range_count - total_count) / range_count) * 100
    
    print(abs_weight_diff < 5, abs_count_diff < 5, constraint, '%0.2f vs %0.2f (%0.2f)' % (range_total_weight, total_weight, abs_weight_diff), '%0.2f vs %0.2f (%0.2f)' % (range_count, total_count, abs_count_diff))
    
print()
    
for constraint in variable_n_indices:
    n_indices = variable_n_indices[constraint].astype(int)
    w_indices = variable_w_indices[constraint].astype(int)
    
    for index, n_index in enumerate(n_indices):
        w_index = w_indices[index]
        
        bucket = index_to_bucket[n_index]
        
        weight = x[w_index] * 10000
        count = x[n_index] * 10000
        avg_weight = weight / count
        valid = (avg_weight > bucket) and (avg_weight < (bucket + 1))
        
        print(valid, constraint, bucket, '%0.3f @ %0.3f' % (avg_weight, count))

In [ ]:
# https://stackoverflow.com/questions/19664865/migrating-from-pulp-to-scipy

import numpy as np
import scipy.optimize as opt

constraints = {
    '0-10': ((2, 5), 213, 826.00),
    '2-3': ((2, 3), 159, 437.80),
    '3-10': ((3, 5), 667, 1667.2),
    '3-4': ((3, 4), 4089, 14856.00),
    '3-6': ((3, 6), 4279, 19030.50),
    '4.5-5.0': ((4, 5), 540, 2553.80),
    '4-5': ((4, 5), 14872, 67174.40),
    '5-6': ((5, 6), 14264, 77238.80),
    '6+': ((6, 8), 618, 4071.80),
    '6-7': ((6, 7), 4640, 29502.20),
    '7-8': ((7, 8), 730, 5323.00),
    '8-9': ((8, 9), 49, 402.20),
    '9+': ((9, 10), 2, 18.30),
}

all_n_indices = []
all_aw_indices = []

variable_n_indices = {}
variable_aw_indices = {}

index_to_bucket = {}

variables_n = { i: [] for i in range(1, 10) }
variables_aw = { i: [] for i in range(1, 10) }

conditions = []
bounds = []
guess = []

counter = 0

all_weights = 0
all_counts = 0

for constraint_name, constraint in constraints.items():
    range_buckets, range_count, range_total_weight = constraint
    
    all_weights = all_weights + range_total_weight / 10000
    all_counts = all_counts + range_count / 10000

    range_count = range_count / 10000
    range_total_weight = range_total_weight / 10000
    
    variable_n_indices[constraint_name] = []
    variable_aw_indices[constraint_name] = []
    
    for bucket in range(range_buckets[0], range_buckets[1]):
        guess_n = range_count * (bucket + 0.5) / (np.sum(range(range_buckets[0], range_buckets[1])) + 0.5 * (range_buckets[1] - range_buckets[0]))
        guess_aw = bucket + 0.5
        
        variable_n_indices[constraint_name].append(counter)
        variables_n[bucket].append(counter)
        all_n_indices.append(counter)
        guess.append(guess_n)
        bounds.append((0, range_count))
        index_to_bucket[counter] = bucket

        counter = counter + 1
        
        variable_aw_indices[constraint_name].append(counter)
        variables_aw[bucket].append(counter)
        all_aw_indices.append(counter)
        guess.append(guess_aw)
        bounds.append((bucket, bucket + 1))
        index_to_bucket[counter] = bucket
        
        counter = counter + 1
    
    variable_n_indices[constraint_name] = np.array(variable_n_indices[constraint_name])
    variable_aw_indices[constraint_name] = np.array(variable_aw_indices[constraint_name])
    
    n_indices = variable_n_indices[constraint_name].astype(int)
    aw_indices = variable_aw_indices[constraint_name].astype(int)
    
#     print(n_indices)
#     print(w_indices)
    
    conditions.append({ 'type': 'ineq', 'fun': lambda x: 0.000001 - np.abs(np.sum(x[n_indices]) - range_count) })
    conditions.append({ 'type': 'ineq', 'fun': lambda x: 0.000001 - np.abs(np.sum(x[aw_indices] * x[n_indices]) - range_total_weight) })
        
#     for index, bucket in enumerate(range(range_buckets[0], range_buckets[1])):
#         n_index = variable_n_indices[constraint_name][index]
#         aw_index = variable_aw_indices[constraint_name][index]
# #         print(bucket, n_index, w_index)

#         x1 = min((bucket + 1) - x[w_index] / x[n_index], min(0, x[w_index] / x[n_index] - bucket))
#         print(x1)
        
# #         conditions.append({ 'type': 'ineq', 'fun': lambda x: min(0.000001, x[w_index] / x[n_index] - bucket)  })
# #         conditions.append({ 'type': 'ineq', 'fun': lambda x: min(0.000001, (bucket + 1) - x[w_index] / x[n_index]) })
#         conditions.append({ 'type': 'ineq', 'fun': lambda x: 0.000001 + x1 })
    
for bucket in variables_n:
    variables_n[bucket] = np.array(variables_n[bucket])
    variables_aw[bucket] = np.array(variables_aw[bucket])
    
    n_indices = variables_n[bucket].astype(int)
    aw_indices = variables_aw[bucket].astype(int)
    
    if len(n_indices) == 0:
        continue
    
#     print(bucket)
#     print('a', n_indices, np.sum(n_indices))
#     print('b', w_indices, np.sum(w_indices))
    
    conditions.append({ 'type': 'ineq', 'fun': lambda x: np.sum(x[aw_indices] * x[n_indices]) - bucket * np.sum(x[n_indices]) })
    conditions.append({ 'type': 'ineq', 'fun': lambda x: (bucket + 1) * np.sum(x[n_indices]) - np.sum(x[aw_indices] * x[n_indices]) })

conditions.append({ 'type': 'ineq', 'fun': lambda x: 0.000001 - np.abs(np.sum(x[all_n_indices]) - all_counts) })
conditions.append({ 'type': 'ineq', 'fun': lambda x: 0.000001 - np.abs(np.sum(x[all_aw_indices] * x[all_n_indices]) - all_weights) })
    
def objective_function(x):
    objective = 0
    
#     for bucket in variables_n:
#         n_indices = variables_n[bucket].astype(int)
#         aw_indices = variables_aw[bucket].astype(int)
        
#         for index, n_index in enumerate(variables_n[bucket]):
#             aw_index = aw_indices[index]
            
#             x1 = bucket - (x[w_index] / x[n_index])
#             x2 = (x[w_index] / x[n_index]) - (bucket + 1)
            
#             print(x1, x2)
#             print(x1, x2, 1 - .5 * (1 + x1 + np.sqrt((x1 - 1) ^ 2 + .01) ) + 1 - .5 * (1 + x2 + np.sqrt((x2 - 1) ^ 2 + .01) ))
            
#             objective = objective + .1 * (1 - .5 * (1 + x1 + np.sqrt((x1 - 1) ** 2 + .01) ) + 1 - .5 * (1 + x2 + np.sqrt((x2 - 1) ** 2 + .01) ))
#             objective = objective + .01 * (-np.log(1 + np.exp(-(-x1))) + -np.log(1 + np.exp(-(-x2))))
#             a = (-np.log(1 + np.exp(-(x[w_index] / x[n_index] - bucket))) + -np.log(1 + np.exp(-((bucket + 1) - x[w_index] / x[n_index]))))
#             if not np.isfinite(a):
#                 print(x[w_index], x[n_index], bucket, x[w_index] / x[n_index] - bucket)
#             print('a', n_indices, n_index, np.sum(x[n_indices]), x[n_index])
#             print('b', w_indices, w_index, np.sum(x[w_indices]), x[w_index])
#             print('c',(x[w_index] / x[n_index] - np.sum(x[w_indices]) / np.sum(x[n_indices])))
    
            # TODO: Remove this as I think it's not correct
#             objective = objective + np.abs((x[w_index] / x[n_index]) - (np.sum(x[w_indices]) / np.sum(x[n_indices])))
#             objective = objective + x[n_index] * np.abs((x[w_index] / x[n_index]) - (np.sum(x[w_indices]) / np.sum(x[n_indices])))
#             objective = objective + np.abs((x[n_index] / x[n_index]) - (np.sum(x[n_indices]) / all_counts))

    for constraint_name, constraint in constraints.items():
        range_buckets, range_count, range_total_weight = constraint
        
        range_count = range_count / 10000
        range_total_weight = range_total_weight / 10000
        
        n_indices = variable_n_indices[constraint_name].astype(int)
        aw_indices = variable_aw_indices[constraint_name].astype(int)
        
        total_subset_count = 0
        
        for n_index in n_indices:
            bucket = index_to_bucket[n_index]
            bucket_indices = variables_n[bucket]
            total_subset_count = total_subset_count + np.sum(x[bucket_indices])
        
        for index, n_index in enumerate(n_indices):
            aw_index = aw_indices[index]
            
            bucket = index_to_bucket[n_index]
            bucket_indices = variables_n[bucket]
            
            objective = objective + np.abs((x[n_index] / np.sum(x[n_indices]) - np.sum(x[bucket_indices]) / total_subset_count))
            #objective = objective + x[n_index] * np.abs((x[n_index] / np.sum(n_indices) - np.sum(x[bucket_indices]) / total_subset_count))
        
        objective = objective + np.abs(np.sum(x[n_indices]) - range_count) / range_count
        objective = objective + np.abs(np.sum(x[aw_indices] * x[n_indices]) - range_total_weight) / range_total_weight
    
    return objective
      
# print(guess)
# print(bounds)
# print(conditions)
print(all_weights, all_counts)
    
results = opt.minimize(objective_function, guess, method='SLSQP', bounds=bounds, constraints = conditions, options={'disp': True})
print(results)

In [ ]:
x = results.x

all_weight = 0
all_count = 0

for bucket in variables_n:
    n_indices = variables_n[bucket].astype(int)
    aw_indices = variables_aw[bucket].astype(int)
    
    total_weight = np.sum(x[aw_indices] * x[n_indices]) * 10000
    total_count = np.sum(x[n_indices]) * 10000
    
    all_weight = all_weight + total_weight
    all_count = all_count + total_count

print('%0.2f vs %0.2f, %0.2f vs %0.2f' % (all_weight / 10000, all_weights, all_count / 10000, all_counts))

print()
    
for bucket in variables_n:
    n_indices = variables_n[bucket].astype(int)
    aw_indices = variables_aw[bucket].astype(int)
    
    total_weight = np.sum(x[aw_indices] * x[n_indices]) * 10000
    total_count = np.sum(x[n_indices]) * 10000
    
    print('%i - %i: %0.2f, %0.2f' % (bucket * 1000, (bucket + 1) * 1000, total_weight / total_count * 1000, total_count / all_count * 100))

print()
    
for constraint in variable_n_indices:
    n_indices = variable_n_indices[constraint].astype(int)
    aw_indices = variable_aw_indices[constraint].astype(int)
    
    total_weight = np.sum(x[aw_indices] * x[n_indices]) * 10000
    total_count = np.sum(x[n_indices]) * 10000
    
    range_buckets, range_count, range_total_weight = constraints[constraint]
    
    abs_weight_diff = np.abs((range_total_weight - total_weight) / range_total_weight) * 100
    abs_count_diff = np.abs((range_count - total_count) / range_count) * 100
    
    print(abs_weight_diff < 5, abs_count_diff < 5, constraint, '%0.2f vs %0.2f (%0.2f)' % (range_total_weight, total_weight, abs_weight_diff), '%0.2f vs %0.2f (%0.2f)' % (range_count, total_count, abs_count_diff))
    
print()
    
for constraint in variable_n_indices:
    range_buckets, range_count, range_total_weight = constraints[constraint]
    
    n_indices = variable_n_indices[constraint].astype(int)
    aw_indices = variable_aw_indices[constraint].astype(int)
    
    subset_bucket_count = 0
    for index, n_index in enumerate(n_indices):
        bucket = index_to_bucket[n_index]
        bucket_n_indices = variables_n[bucket].astype(int)
        bucket_count = np.sum(x[bucket_n_indices]) * 10000
        subset_bucket_count = subset_bucket_count + bucket_count
    
    for index, n_index in enumerate(n_indices):
        aw_index = aw_indices[index]
        
        bucket = index_to_bucket[n_index]
        
        bucket_n_indices = variables_n[bucket].astype(int)
        bucket_count = np.sum(x[bucket_n_indices]) * 10000
        
        weight = x[aw_index]
        count = x[n_index] * 10000
        avg_weight = weight
        valid = (avg_weight > bucket) and (avg_weight < (bucket + 1))
        
        print(valid, constraint, bucket, '%0.3f @ %0.3f (%0.3f vs %0.2f)' % (avg_weight, count, count / range_count * 100, bucket_count / subset_bucket_count * 100))